# Build your comparative framework!

In [6]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import os 
import pandas as pd
import json

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
from clutils.nbutils.params import get_param_ranges

In [4]:
pal = sns.color_palette("Paired", 11)
dict_color = {"random":pal[0], "distance":pal[1], "pagerank":pal[2], "sa": pal[3], "ig": pal[4], "gradcam": pal[5], "occlusion":pal[6], 
              "basic_gnnexplainer": pal[7], "gnnexplainer": pal[8], 
              "pgmexplainer":pal[9], "subgraphx": pal[10]}


In [5]:
sns.set()
sns.set_context("notebook", rc={"legend.fontsize":20, "legend.title_fontsize":20, 
                                "axes.titlesize":19,"axes.labelsize":17,
                               "xtick.labelsize" : 16, "ytick.labelsize" : 16})


In [6]:
dir = '/cluster/home/kamara/Explain/csv/node_classification/syn'

jsonpath = f'/cluster/home/kamara/Explain/configs/config_syn_sparsity_avg10expe.json'
ranges = get_param_ranges(jsonpath)

df_sparsity = pd.read_csv(os.path.join(dir, f'nc_syn_sparsity_avg10expe.csv'), index_col = 0)
df_sparsity = df_sparsity.groupby(by=['dataset', 'sparsity', 'explainer_name', 'true_label_as_target', 'hard_mask']).mean()
df_sparsity = df_sparsity.reset_index()

In [7]:
def explainer_type(row):
    if row['explainer_name'] in ['basic_gnnexplainer', 'gnnexplainer', 'pgmexplainer', 'occlusion', 'subgraphx'] :
      return('pertubation')
    elif row['explainer_name'] in ['sa', 'ig', 'gradcam'] :
      return('gradient')
    return('baseline')

df_sparsity['type'] = df_sparsity.apply(lambda row: explainer_type(row), axis=1)

In [8]:
df_sparsity["1-fidelity_acc-"] = 1- df_sparsity["fidelity_acc-"]
df_sparsity["1-fidelity_prob-"] = 1- df_sparsity["fidelity_prob-"]
df_sparsity['charact_prob'] = 2 * df_sparsity['fidelity_prob+'] * (1-df_sparsity['fidelity_prob-'])/ (df_sparsity['fidelity_prob+']  + 1-df_sparsity['fidelity_prob-'])
df_sparsity['charact_acc'] = 2 * df_sparsity['fidelity_acc+'] * (1-df_sparsity['fidelity_acc-'])/ (df_sparsity['fidelity_acc+']  + 1-df_sparsity['fidelity_acc-'])


In [9]:
df_sparsity["1-fidelity_gnn_acc-"] = 1- df_sparsity["fidelity_gnn_acc-"]
df_sparsity["1-fidelity_gnn_prob-"] = 1- df_sparsity["fidelity_gnn_prob-"]
df_sparsity['charact_gnn_prob'] = 2 * df_sparsity['fidelity_gnn_prob+'] * (1-df_sparsity['fidelity_gnn_prob-'])/ (df_sparsity['fidelity_gnn_prob+']  + 1-df_sparsity['fidelity_gnn_prob-'])
df_sparsity['charact_gnn_acc'] = 2 * df_sparsity['fidelity_gnn_acc+'] * (1-df_sparsity['fidelity_gnn_acc-'])/ (df_sparsity['fidelity_gnn_acc+']  + 1-df_sparsity['fidelity_gnn_acc-'])


In [10]:
df_sparsity = df_sparsity.reset_index(drop=True)
df_sparsity

,dataset,sparsity,explainer_name,true_label_as_target,hard_mask,num_test,fidelity_acc+,fidelity_acc-,fidelity_prob+,fidelity_prob-,...,non_zero_values_init,type,1-fidelity_acc-,1-fidelity_prob-,charact_prob,charact_acc,1-fidelity_gnn_acc-,1-fidelity_gnn_prob-,charact_gnn_prob,charact_gnn_acc
0,ba_house,0.000,basic_gnnexplainer,False,False,100.0,-1.000,-1.000,-1.000000,-1.000000,...,240.332,pertubation,2.000,2.000000,-4.000000,-4.000000,0.554,0.607394,0.545426,0.546910
1,ba_house,0.000,basic_gnnexplainer,False,True,100.0,-1.000,-1.000,-1.000000,-1.000000,...,240.332,pertubation,2.000,2.000000,-4.000000,-4.000000,1.000,1.000000,0.828921,0.883305
2,ba_house,0.000,basic_gnnexplainer,True,False,100.0,0.569,0.407,0.451042,0.295797,...,261.956,pertubation,0.593,0.704203,0.549884,0.580752,2.000,2.000000,-4.000000,-4.000000
3,ba_house,0.000,basic_gnnexplainer,True,True,100.0,0.800,0.000,0.654603,0.000000,...,261.956,pertubation,1.000,1.000000,0.791251,0.888889,2.000,2.000000,-4.000000,-4.000000
4,ba_house,0.000,distance,False,False,100.0,-1.000,-1.000,-1.000000,-1.000000,...,4058.193,baseline,2.000,2.000000,-4.000000,-4.000000,0.392,0.438536,0.482939,0.474506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,ba_bottle,0.997,random,True,True,100.0,0.132,0.605,0.129106,0.564257,...,3943.120,baseline,0.395,0.435743,0.199193,0.197875,2.000,2.000000,-4.000000,-4.000000
716,ba_bottle,0.997,sa,False,False,100.0,-1.000,-1.000,-1.000000,-1.000000,...,3783.328,gradient,2.000,2.000000,-4.000000,-4.000000,0.373,0.403726,0.012793,0.007915
717,ba_bottle,0.997,sa,False,True,100.0,-1.000,-1.000,-1.000000,-1.000000,...,3783.328,gradient,2.000,2.000000,-4.000000,-4.000000,0.373,0.402472,0.012936,0.005952
718,ba_bottle,0.997,sa,True,False,100.0,0.001,0.604,0.001100,0.564071,...,3773.754,gradient,0.396,0.435929,0.002194,0.001995,2.000,2.000000,-4.000000,-4.000000


In [11]:
df = df_sparsity

In [12]:
sparsity_slider = widgets.SelectionSlider(
    options=['0', '0.7', '0.99', '0.997'],
    value='0',
    description='Sparsity:',
    disabled=False,
)



focus_select = widgets.ToggleButtons(
    options=[('Phenomenon', True), ('GNN model', False)],
    value = True,
    description='Focus:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Explain phenomenon', 'Explain GNN logic'],
#     icons=['check'] * 3
)

hard_mask_select = widgets.ToggleButtons(
    options=[('Hard', True), ('Soft', False)],
    value = True,
    description='Nature:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Mask has binary values', 'Mask has scalar values'],
#     icons=['check'] * 3
)

charact_type = widgets.ToggleButtons(
    options=[('Closeness', True), ('Reproducibility', False)],
    value = True,
    description='Characterization:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Characterization on probability', 'Characterization on accuracy'],
#     icons=['check'] * 3
)


dataset_select = widgets.Dropdown(
    options=["ba_house", "ba_grid", "tree_cycle", "tree_grid", "ba_bottle"],
    value='ba_house',
    description='Dataset:',
    disabled=False,
)


explainer_select = widgets.Dropdown(
    options=["random",
      "distance",
      "pagerank",
      "sa",
      "ig",
      "occlusion",
      "basic_gnnexplainer",
      "gnnexplainer",
      "pgmexplainer",
            "subgraphx"],
    value='random',
    description='Explainer:',
    disabled=False,
)


In [13]:
def reduce(df, **kwargs):
    if 'focus_phen' in kwargs:
        focus_phen = kwargs.get('focus_phen')
        df = df.loc[df['true_label_as_target'] == focus_phen]
        prob = kwargs.get('prob') if 'prob' in kwargs else True
        if focus_phen:
            df.loc[:, "charact"] = df.loc[:,"charact_prob"] if prob else df.loc[:,"charact_acc"]
            df.loc[:, "fid+"] = df.loc[:,"fidelity_prob+"] if prob else df.loc[:,"fidelity_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:,"1-fidelity_prob-"] if prob else df.loc[:,"1-fidelity_acc-"]
        else:
            df.loc[:, "charact"] = df.loc[:, "charact_gnn_prob"] if prob else df.loc[:, "charact_gnn_acc"]
            df.loc[:, "fid+"] = df.loc[:, "fidelity_gnn_prob+"] if prob else df.loc[:, "fidelity_gnn_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:, "1-fidelity_gnn_prob-"] if prob else df.loc[:, "1-fidelity_gnn_acc-"]
    
    if 'hard_mask' in kwargs:
        df = df.loc[df['hard_mask'] == kwargs.get('hard_mask')]
    if 'sparsity' in kwargs:
        df = df.loc[df['sparsity'] == eval(kwargs.get('sparsity'))]
    if 'dataset' in kwargs:
        df = df.loc[df['dataset'] == kwargs.get('dataset')]
    if 'explainer' in kwargs:
        df = df.loc[df['explainer_name'] == kwargs.get('explainer')]
    return df

In [42]:
@interact
def score(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity)
    df_char = pd.pivot_table(dfs, values='charact', index=['explainer_name'], columns='dataset')
    df_char['mean_charact'] = df_char.mean(axis=1)
    
    res = df_char.reset_index().rename_axis(None, axis=1)
    df_char = df_char.sort_values(by=['mean_charact'])
    
    sub = dfs.groupby('explainer_name').mean().reset_index().rename_axis(None, axis=1)
    sub = sub[["explainer_name", "mask_size_transf", "mask_entropy_transf", "max_avg_transf", '1-fid-', 'fid+', 'charact']]
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, sharey=True, figsize=(21, 10))
    
    sns.barplot(x="1-fid-", y="explainer_name", data=sub, palette=dict_color, ax=ax1)
    ax1.set(xlabel=None)
    ax1.set(ylabel="Performance")
    ax1.set(title="Causal")
    
    sns.barplot(x="fid+", y="explainer_name", data=sub, palette=dict_color, ax=ax2)
    ax2.set(xlabel=None)
    ax2.set(ylabel=None)
    ax2.set(title="Counterfactual")
    
    sns.barplot(x="charact", y="explainer_name", data=sub, palette=dict_color, ax=ax3)
    ax3.set(xlabel=None)
    ax3.set(ylabel=None)
    ax3.set(title="Characterisation") 
    
    sns.barplot(x="mask_size_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax4)
    ax4.set(xlabel=None)
    ax4.set(ylabel="Properties")
    ax4.set(title="Selective")
    
    sns.barplot(x="mask_entropy_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax5)
    ax5.set(xlabel=None)
    ax5.set(ylabel=None)
    ax5.set(title="Discriminative")
    
    sns.barplot(x="max_avg_transf", y="explainer_name", data=sub, palette=dict_color, ax=ax6)
    ax6.set(xlabel=None)
    ax6.set(ylabel=None)
    ax6.set(title="Confident") 
    
    fig.tight_layout()
    return

interactive(children=(ToggleButtons(description='Focus:', index=1, options=(('Phenomenon', True), ('GNN model'…

In [37]:
# Mask properties
@interact
def mask_properties(dataset=dataset_select, focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type, sparsity=sparsity_slider):
    df_mask = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset)
    df_mask = df_mask[["explainer_name", "mask_size_transf", "mask_entropy_transf", "max_avg_transf", "charact"]]
    df_mask = df_mask.sort_values(by=['charact'])
    return df_mask.set_index('explainer_name')

interactive(children=(Dropdown(description='Dataset:', options=('ba_house', 'ba_grid', 'tree_cycle', 'tree_grid', 'ba_bottle'), value…

In [14]:
@interact
def get_explanation(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type, dataset=dataset_select, explainer=explainer_select):
    df_unit = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset, explainer=explainer)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    g = sns.barplot(x="value", y="variable", data=pd.melt(df_unit[['1-fid-', 'fid+', 'charact']]), palette=sns.color_palette("tab10"))
    g.set_yticklabels(['causal','counterfactual', 'characterization'])
    g.set(xlabel=None)
    g.set(ylabel=None)
    g.set_xlim(0,1)
    g.set(title="Type of explanation")
    
    
    textstr = '\n'.join((
    r'$\mathrm{selective}=%.0f$' % (df_unit['mask_size_transf'], ),
    r'$\mathrm{discriminative}=%.2f$' % (df_unit['mask_entropy_transf'], ),
    r'$\mathrm{confident}=%.2f$' % (df_unit['max_avg_transf'], )))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    g.text(1.05, 0.4, textstr, transform=g.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)
    return


interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…

In [38]:
@interact
def get_scatterplots_expl(focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    g = sns.relplot(
        data=dfs,
        x="fid+", y="1-fid-", col="sparsity", row="dataset", 
        hue="explainer_name", palette=dict_color, s=140, style="type", alpha =1, edgecolor="grey"
    )
    
    plt.xlim(0, 1)
    plt.ylim(0, 1.2)
    for lh in g._legend.legendHandles: 
        lh.set_alpha(1)
        lh._sizes = [130] 

interactive(children=(ToggleButtons(description='Focus:', index=1, options=(('Phenomenon', True), ('GNN model'…